In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sbn
import pickle as pkl 

pd.set_option('display.max_columns', 100)

6/7/21: I'm going to hold off on using genetic data for now - with the new formulation for inferring the "functional state" of genes based on the reversed graph edges much of the relevant genetic information may be inferred using expression data. That said, it is likely that mutation data will be critical in predicting longitudinal/perturbation data - if a gene state can be inactive (at the unperturbed time point) it could later become activated if it's genetically functional; however, it couldn't become later functional if it's genetically inactive... etc etc. tldr; genetic information important, but expression good start. 

In [7]:
mut = pd.read_csv('../data/raw/ccle_mutation.txt')
mut.head()

C:\Users\natha\miniconda3\envs\gcnvae\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (3,19,22,27,28,29,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,dbSNP_RS,dbSNP_Val_Status,Genome_Change,Annotation_Transcript,DepMap_ID,cDNA_Change,Codon_Change,Protein_Change,isDeleterious,isTCGAhotspot,TCGAhsCnt,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,Variant_annotation,CGA_WES_AC,HC_AC,RD_AC,RNAseq_AC,SangerWES_AC,WGS_AC
0,VPS13D,55187,37,1,12359347,12359347,+,Nonsense_Mutation,SNP,C,A,NaN,NaN,g.chr1:12359347C>A,ENST00000358136.3,ACH-000001,c.6122C>A,c.(6121-6123)tCa>tAa,p.S2041*,True,False,NaN,False,0.0,NaN,damaging,34:213,NaN,NaN,NaN,34:221,NaN
1,AADACL4,343066,37,1,12726308,12726322,+,In_Frame_Del,DEL,CTGGCGTGACGCCAT,-,rs58218425|rs139261871|rs369427733|rs560787141,byFrequency,g.chr1:12726308_12726322delCTGGCGTGACGCCAT,ENST00000376221.1,ACH-000001,c.786_800delCTGGCGTGACGCCAT,c.(784-801)tcctggcgtgacgccatc>tcc,p.WRDAI263del,False,False,NaN,False,3.0,NaN,other non-conserving,57:141,NaN,NaN,NaN,9:0,28:32
2,IFNLR1,163702,37,1,24484172,24484172,+,Silent,SNP,G,A,NaN,NaN,g.chr1:24484172G>A,ENST00000327535.1,ACH-000001,c.1011C>T,c.(1009-1011)ggC>ggT,p.G337G,False,False,NaN,False,0.0,NaN,silent,118:0,NaN,NaN,10:0,118:0,18:0
3,TMEM57,55219,37,1,25785018,25785019,+,Frame_Shift_Ins,INS,-,A,NaN,NaN,g.chr1:25785018_25785019insA,ENST00000374343.4,ACH-000001,c.789_790insA,c.(790-792)aaafs,p.K264fs,True,False,0.0,False,0.0,NaN,damaging,NaN,NaN,NaN,6:28,NaN,NaN
4,ZSCAN20,7579,37,1,33954141,33954141,+,Missense_Mutation,SNP,T,G,NaN,NaN,g.chr1:33954141T>G,ENST00000361328.3,ACH-000001,c.494T>G,c.(493-495)gTg>gGg,p.V165G,False,False,NaN,False,0.0,NaN,other non-conserving,28:62,NaN,NaN,NaN,27:61,NaN


In [16]:
info = pd.read_csv('../data/raw/ccle_info.txt')
info.head()

,DepMap_ID,cell_line_name,stripped_cell_line_name,CCLE_Name,Alias,COSMICID,sex,source,Achilles_n_replicates,cell_line_NNMD,culture_type,culture_medium,cas9_activity,RRID,WTSI_Master_Cell_ID,sample_collection_site,primary_or_metastasis,primary_disease,Subtype,age,Sanger_Model_ID,depmap_public_comments,lineage,lineage_subtype,lineage_sub_subtype,lineage_molecular_subtype
0,ACH-000001,NIH:OVCAR-3,NIHOVCAR3,NIHOVCAR3_OVARY,OVCAR3,905933.0,Female,ATCC,NaN,NaN,NaN,NaN,NaN,CVCL_0465,2201.0,ascites,Metastasis,Ovarian Cancer,"Adenocarcinoma, high grade serous",60.0,SIDM00105,NaN,ovary,ovary_adenocarcinoma,high_grade_serous,NaN
1,ACH-000002,HL-60,HL60,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,905938.0,Female,ATCC,NaN,NaN,NaN,NaN,NaN,CVCL_0002,55.0,haematopoietic_and_lymphoid_tissue,Primary,Leukemia,"Acute Myelogenous Leukemia (AML), M3 (Promyelo...",35.0,SIDM00829,NaN,blood,AML,M3,NaN
2,ACH-000003,CACO2,CACO2,CACO2_LARGE_INTESTINE,"CACO2, CaCo-2",NaN,Male,ATCC,NaN,NaN,NaN,NaN,NaN,CVCL_0025,NaN,Colon,NaN,Colon/Colorectal Cancer,Adenocarcinoma,NaN,SIDM00891,NaN,colorectal,colorectal_adenocarcinoma,NaN,NaN
3,ACH-000004,HEL,HEL,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,907053.0,Male,DSMZ,2.0,-3.079202,Suspension,RPMI + 10% FBS,52.4,CVCL_0001,783.0,haematopoietic_and_lymphoid_tissue,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",30.0,SIDM00594,NaN,blood,AML,M6,NaN
4,ACH-000005,HEL 92.1.7,HEL9217,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Male,ATCC,2.0,-2.404409,Suspension,RPMI + 10% FBS,86.6,CVCL_2481,NaN,bone_marrow,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",30.0,SIDM00593,NaN,blood,AML,M6,NaN


In [19]:
lincs_cells = pd.read_csv('../../LINCS/LVL3/data/raw/cellinfo_beta.txt', sep='\t')
lincs_cells.head()

,cell_iname,cellosaurus_id,donor_age,donor_age_death,donor_disease_age_onset,doubling_time,growth_medium,provider_catalog_id,feature_id,cell_type,donor_ethnicity,donor_sex,donor_tumor_phase,cell_lineage,primary_disease,subtype,provider_name,growth_pattern,ccle_name,cell_alias
0,1HAE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal,Unknown,Unknown,Unknown,unknown,unknown,normal fibroblast sample,NaN,unknown,NaN,NaN
1,AALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal,Unknown,Unknown,Unknown,unknown,unknown,normal epithelium sample,NaN,unknown,NaN,NaN
2,AG06263_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tumor,Unknown,Unknown,Unknown,unknown,unknown,unknown,NaN,unknown,NaN,NaN
3,AG06840_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tumor,Unknown,Unknown,Unknown,unknown,unknown,unknown,NaN,unknown,NaN,NaN
4,AG078N1_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tumor,Unknown,Unknown,Unknown,unknown,unknown,unknown,NaN,unknown,NaN,NaN


In [24]:
lincs_cells.shape

(240, 20)

In [2]:
with open('../../PPI_graph.pkl', 'rb') as f: 
    PPI_graph = pkl.load(f)

In [4]:
PPI_graph.keys()

dict_keys(['nodelist', 'graphs', 'genemap', 'meta', 'creation_date'])

In [9]:
PPI_mut_overlap = set(PPI_graph['nodelist']).intersection(set(mut.Hugo_Symbol))
len(PPI_mut_overlap)

12726

In [23]:
# Step 1: what cell lines are in LINCS? 
Lcells = lincs_cells[['cell_iname', 'ccle_name']].merge(info[['CCLE_Name', 'DepMap_ID']], left_on='ccle_name', right_on='CCLE_Name')
Lcells.shape

(138, 4)